In [1]:
%load_ext autoreload

In [2]:
%aimport model

In [3]:
import skorch

In [4]:
import numpy as np

In [5]:
import keras

Using TensorFlow backend.


In [6]:
import torch

In [7]:
dataset = 'fashion_mnist'

In [8]:
dataset_config = {
    'size': 32,
    'channels': 1,
    'classes': 10
}

In [9]:
net = skorch.net.NeuralNetClassifier(
    module=model.WideResNet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=12,
        widen_factor=5,
    ),
    batch_size=32,
    max_epochs=10,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/wrn{last_epoch[epoch]}.pt", monitor=None))
    ],
    device='cuda',
    train_split=None, # Workaround for https://github.com/dnouri/skorch/issues/218
    lr=1e-04,
)
net.initialize()

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): Conv2d(16, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv_transform): Conv2d(16, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (1): ResidualBlock(
            (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           

In [10]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [11]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [ ]:
net.fit_loop(x_train, y_train)

Checkpoint! Saving model to models/wrn1.pt.
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4319  91.6270
Checkpoint! Saving model to models/wrn2.pt.
      2        0.2350  94.8131
Checkpoint! Saving model to models/wrn3.pt.
      3        0.1803  95.1120
Checkpoint! Saving model to models/wrn4.pt.
      4        0.1356  94.9487
Checkpoint! Saving model to models/wrn5.pt.
      5        0.1011  95.7791
Checkpoint! Saving model to models/wrn6.pt.
      6        0.0763  94.9037
Checkpoint! Saving model to models/wrn7.pt.
      7        0.0622  95.6675
Checkpoint! Saving model to models/wrn8.pt.
      8        0.0499  95.0206


In [ ]:
prediction_test = net.predict(x_test)

In [ ]:
(prediction_test == y_test).mean()